In [ ]:
/utils
pi:acos -1
atan2:{2*atan x%sqrt[sqr[x]+sqr y]+y}
sqr:{x*x}
getDoy:{1+x-"d"$1+(-).`month`mm$\:x}
times:{[int;start]select from([]dt:"p"$s+`minute$til`int$60*24*1+("z"$edate)-s:"z"$start)where 0=i mod int}

### Chain data

In [ ]:
chainUrl:"ftp://chain.physics.unb.ca/gps/ismr"
chainStn:`arv`arc`chu`cor`edm`fsi`fsm`gil`gjo`gri`mcm`rab`ran`rep
sdate:2015.01.01
edate:2017.12.31
user:"kibebuy@gmail.com"
pass:"4Kindahafti4"

In [ ]:
chainCol:raze(31#"S";csv)0:`:chaincols.csv
chainLatLong:1!update long-360 from("SFF";(),csv)0:`:./cslatlong.csv

In [ ]:
fileArgs:(cross/)(chainStn;sdate+til 1+edate-sdate;til 24)

loadChain:{[stn;dt;hr]
  url:0N!"/"sv(chainUrl;yr;sdoy;-2#"0",string hr;string[stn],"c",(-2#yr:string`year$dt),(sdoy:-3#"00",string doy:getDoy dt),(.Q.a hr),".ismr.gz");
  cmd:"curl -u ",user,":",pass," ",url," 2>/dev/null | gunzip -c 2>/dev/null";
  if[(::)~r:@[system;cmd;{[e] -2"Error: ",e;}];:()];
  update dt,doy,cs:stn from t:flip chainCol!(31#"F";csv)0:r}

start:.z.T;
chain:raze loadChain .'fileArgs
-1"\nReading in chain data took ",string .z.T-start;

In [ ]:
chain:select from chain where elevation>=30,locktimeSig1>200

In [ ]:
chain:update dt+"v"$tow mod 86400 from chain lj chainLatLong
chain:update tec*sqrt 1-sqr cos[elevation*pi%180]*6378.137%6378.137+110 from chain
chain:update s4*sin[elevation*pi%180]xexp .9 from chain
chain:update sigPhiVer*sqrt sin elevation*pi%180 from chain

chain:0!select med tec,med dtec,med s4,med specSlope,med SI,med sigPhiVer by dt,doy,cs from chain

In [ ]:
chain:{[t;hr]
   t lj 2!?[t;();0b;(`dt`cs,`$"sigPhiVer",string[hr],"hr")!((-;`dt;"u"$60*hr);`cs;`sigPhiVer)]
  }/[chain;1 3 6 9 12 24]

### Solar data

In [ ]:
solarUrl:"ftp://spdf.gsfc.nasa.gov/pub/data/omni/"
dir5mn:"high_res_omni/omni_5min"
dir1hr:"low_res_omni/omni2_"
/sdate:2015.01.01
/edate:2017.12.31

In [ ]:
solar5mn:solarUrl,dir5mn
solar1hr:solarUrl,dir1hr

omniDate:2014.01.01
y:2014+til 4
width5mn:4 4 3 3 3 3 4 4 4 7 7 6 7 8 8 8 8 8 8 8 8 8 8 8 8 7 9 6 7 7 6 8 8 8 8 8 8 6 6 6 6 6 6 6 7 5 9 9 9
width1hr:4 4 3 5 3 3 4 4 6 6 6 6 6 6 6 6 6 6 6 6 6 6 9 6 6 6 6 6 6 9 6 6 6 6 6 7 7 6 3 4 6 5 10 9 9 9 9 9 3 4 6 6 6 6 5
dtype5mn:"IIIIIIIIIIIFIFFFFFFFFFFFFFFFFFFFFFFFFIFIIIIIFFFFF"
dtype1hr:"IIIIIIIIFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFIIFIFFFFFFFIFFFIF"

In [ ]:
col5mn:raze(49#"S";csv)0:`:solarcols.csv
col1hr:raze(55#"S";csv)0:`:solarcols1hr.csv

In [ ]:
loadSolar:{[y;t]
  input:`width`dtype`url`file`col!$[t=5;(width5mn;dtype5mn;solar5mn;".asc";col5mn);(width1hr;dtype1hr;solar1hr;".dat";col1hr)];
  0N!url:input[`url],string[y],input`file;
  cmd:"curl ",url," 2>/dev/null";
  if[(::)~r:@[system;cmd;{[e] -2"Error: ",e;}];:()];
  flip input[`col]!(input`dtype;input`width)0:r
  }

start:.z.T
solar5mn:raze loadSolar[;5]each y
solar1hr:raze loadSolar[;1]each -3#y
-1"\nReading in solar data took ",string .z.T-start;

In [ ]:
solar5mn:update angle:atan2[By;Bz],btot:sqrt sqr[By]+sqr Bz from solar5mn
solar5mn:update borovsky:0.0329*sqr[V*sin angle%2]*sqrt[n]*xexp[mach;-.18]*exp sqrt mach%3.42 from solar5mn
solar5mn:update angle:angle+pi from solar5mn where i in where 0>=cos[angle]*Bz*btot
solar5mn:update newell:xexp[V;1.33333]*xexp[abs sin angle%2;2.66667]*xexp[btot;0.66667] from solar5mn

solar5mn:select dt:times[5;omniDate]`dt,By,Bz,AE,symH,V,P,borovsky:borovsky,newell:newell,proton10,proton30,proton60 from solar5mn
solar5mn:reverse fills reverse times[1;omniDate]lj 1!solar5mn

solar5mn:{[t;lb]
   t lj 1!?[t;();0b;(`dt,`$string[c],\:"_",string lb)!(enlist[(+;`dt;"u"$lb)],c:`By`Bz`AE`symH`V`P`borovsky`newell)]
  }/[solar5mn;15 30]

In [ ]:
solar1hr:select dt:times[60;sdate]`dt,f107,kp:kp%10 from solar1hr

In [ ]:
goesUrl:"https://satdat.ngdc.noaa.gov/sem/goes/data/full"
fileArgs:5#sdate+til 1+edate-sdate

loadGoes:{[dt]
  url:0N!"/"sv(goesUrl;string`year$dt;-2#"0",string`mm$dt;"goes15/csv";"g15_xrs_2s_",d,"_",(d:ssr[string dt;".";""]),".csv");  
  cmd:"curl ",url," 2>/dev/null";
  if[(::)~r:@[system;cmd;{[e] -2"Error: ",e;}];:()];
  t:select dt:first time_tag.datetime,time:first time_tag.time,minute:first time_tag.minute,goes:avg B_FLUX by 1 xbar time_tag.minute from("ZSJFSJF";(),csv)0:139_r;
  select dt,goes from update dt:dt-time-minute from t
  }

start:.z.T
goesTab:raze loadGoes each fileArgs
-1"\nReading in GOES data took ",string .z.T-start;

In [ ]:
solar:(lj/)[solar5mn;1!/:(solar1hr;goesTab)]
solar:select from solar where dt within(2015.01.01;edate)

### Magnetometer data

In [ ]:
magUrl:"http://data.carisma.ca/FGM/1Hz/"
magStn:`mcmu`fsim`fchu!`mcm`fsi`chu

In [ ]:
fileArgs:5#key[magStn]cross sdate+til 1+edate-sdate;

loadMag:{[s;dt]
  0N!url:magUrl,sv["/";ssr[string dt;"."]each("/";"")],upper[string s],".F01.gz";
  cmd:"curl ",url," 2>/dev/null | gunzip -c 2>/dev/null";
  if[(::)~r:@[system;cmd;{[e] -2"Error: ",e;}];:()];
  d:ssr[;"  ";" "]each r;
  t:update{(+)."DV"$'0 8_x}each dt from flip`dt`x`y`z!("*FFF";" ")0:1_d;
  update cs:magStn s from 0!select first x,first y,first z by("n"$"u"$1)xbar"p"$dt from t
 }

start:.z.T;
mag:raze loadMag .'fileArgs
-1"\nReading in mag data took ",string .z.T-start;

In [ ]:
chain:5#chain
solar:5#solar
mag:5#mag

### Save data

In [ ]:
dstdir:`:/SpaceWeather/test

In [ ]:
savechain:{[dir;t;d]0N!.Q.par[dir;d;`$"chain/"]set .Q.en[dir]select from t where d="d"$dt}
savechain[dstdir;chain]each exec distinct"d"$dt from chain;
.Q.chk dstdir;

In [ ]:
savesolar:{[dir;t;d]0N!.Q.par[dir;d;`$"solar/"]set .Q.en[dir]select from t where d="d"$dt}
savesolar[dstdir;solar]each exec distinct"d"$dt from solar;
.Q.chk dstdir;

In [ ]:
savemag:{[dir;t;d]0N!.Q.par[dir;d;`$"mag/"]set .Q.en[dir]select from t where d="d"$dt}
savemag[dstdir;mag]each exec distinct"d"$dt from mag;
.Q.chk dstdir;